# Importing necessary libraries to run models

In [1]:
import numpy as np
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from pickle import load
from keras.models import load_model

Using TensorFlow backend.


# Need to load my 5 txt files to make the 5 models

In [2]:
def loader(textfile):
    # Open the file as read-only
    file = open(textfile, 'r')
    # Actually read the file
    text = file.read()
    # Now that I have the data in my text variable, need to close out of the file
    file.close()
    return text

In [3]:
# Create the 5 instances of the 5 ratings

one = loader('One_Star.txt')
# two = loader('Two_Star.txt')
# three = loader('Three_Star.txt')
# four = loader('Four_Star.txt')
# five = loader('Five_Star.txt')

In [4]:
# Need to split up each of these data sets into training sequences by the different lines. 
# End goal is to have user input 5 words and the text generator will work its magic

In [5]:
one_lines = one.split('\n')
# two_lines = two.split('\n')
# three_lines = three.split('\n')
# four_lines = four.split('\n')
# five_lines = five.split('\n')

In [6]:
# Each line has been broken out with 5 sequence of words
one_lines[:5]

['atrocious the live music was loud',
 'the live music was loud and',
 'live music was loud and mediocre',
 'music was loud and mediocre the',
 'was loud and mediocre the decor']

# Need to convert the words in my data to integers in order to be used by the model. 

Will need to train the keras tokenizer on entire dataset to assign a unique ID to each unique word in the dataset

In [7]:
tokenizer = Tokenizer()

def word_to_int(lines):
    tokenizer.fit_on_texts(lines)
    sequences = tokenizer.texts_to_sequences(one_lines)
    # Need the total size of the vocabulary for our embedding layer in model
    vocab_size = len(tokenizer.word_index) + 1
    return sequences, vocab_size

In [8]:
# Running the function for all 5 data sets
one_sequence, one_vocab_size = word_to_int(one_lines)
# two_sequence, two_vocab_size = word_to_int(two_lines)
# three_sequence, three_vocab_size = word_to_int(three_lines)
# four_sequence, four_vocab_size = word_to_int(four_lines)
# five_sequence, five_vocab_size = word_to_int(five_lines)

In [9]:
# each line has been converted to unique ints to represent each word
one_sequence[:5]

[[2374, 1, 551, 549, 7, 809],
 [1, 551, 549, 7, 809, 3],
 [551, 549, 7, 809, 3, 1044],
 [549, 7, 809, 3, 1044, 1],
 [7, 809, 3, 1044, 1, 1081]]

In [10]:
part_one = one_sequence[0:200000]

part_two = one_sequence[200001:400000]

part_three = one_sequence[400001:600000]

part_four = one_sequence[600001:800000]

part_five = one_sequence[800001:1000000]

part_six = one_sequence[1000001:1200000]

part_seven = one_sequence[1200001:-1]

# Need to convert my sequences into inputs and outputs to structure my model to predict words based on previous set of words

## One Star Review Modeling

In [11]:
one_sequence = np.array(part_one)
X, y = one_sequence[:,:-1], one_sequence[:,-1]
y = to_categorical(y, num_classes=one_vocab_size)
seq_length = X.shape[1]

In [14]:
# defining my model

model = Sequential()
model.add(Embedding(one_vocab_size, 50, input_length=seq_length))
model.add(LSTM(25, return_sequences=True))
model.add(LSTM(25))
model.add(Dense(25, activation='relu'))
model.add(Dense(one_vocab_size, activation = 'softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 5, 50)             1883750   
_________________________________________________________________
lstm_3 (LSTM)                (None, 5, 25)             7600      
_________________________________________________________________
lstm_4 (LSTM)                (None, 25)                5100      
_________________________________________________________________
dense_3 (Dense)              (None, 25)                650       
_________________________________________________________________
dense_4 (Dense)              (None, 37675)             979550    
Total params: 2,876,650
Trainable params: 2,876,650
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X,y,batch_size=128, epochs = 50)

Epoch 1/50
200000/200000 [==============================] - 244s 1ms/step - loss: 6.9837 - acc: 0.0470
Epoch 2/50
200000/200000 [==============================] - 261s 1ms/step - loss: 6.5298 - acc: 0.0526
Epoch 3/50
200000/200000 [==============================] - 269s 1ms/step - loss: 6.3249 - acc: 0.0655
Epoch 4/50
200000/200000 [==============================] - 262s 1ms/step - loss: 6.1356 - acc: 0.0808
Epoch 5/50
200000/200000 [==============================] - 254s 1ms/step - loss: 5.9557 - acc: 0.0958
Epoch 6/50
200000/200000 [==============================] - 255s 1ms/step - loss: 5.8238 - acc: 0.1027
Epoch 7/50
200000/200000 [==============================] - 237s 1ms/step - loss: 5.7190 - acc: 0.1087
Epoch 8/50
200000/200000 [==============================] - 235s 1ms/step - loss: 5.6188 - acc: 0.1155
Epoch 9/50
200000/200000 [==============================] - 234s 1ms/step - loss: 5.5250 - acc: 0.1219
Epoch 10/50
200000/200000 [==============================] - 235s 1ms/ste

In [16]:
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [17]:
# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

In [18]:
def word_to_int(lines):
#     tokenizer.fit_on_texts(lines)
    sequences = tokenizer.texts_to_sequences(one_lines)
    # Need the total size of the vocabulary for our embedding layer in model
#     vocab_size = len(tokenizer.word_index) + 1
    return sequences

In [19]:
one_sequence = np.array(part_two)
X, y = one_sequence[:,:-1], one_sequence[:,-1]
y = to_categorical(y, num_classes=one_vocab_size)
seq_length = X.shape[1]

In [20]:
model.fit(X,y,batch_size=128, epochs = 1)

Epoch 1/1
 22144/399999 [>.............................] - ETA: 6:36 - loss: 6.5524 - acc: 0.0689

KeyboardInterrupt: 